
# 🚗 Day 05 — Model Selection & Hyperparameter Tuning

**What you'll learn (with notes):**
- Reuse Day 03/04 artifacts, keep transformations in **pipelines** (no leakage ✅).
- Compare multiple models fairly using **cross‑validation**.
- Tune hyperparameters with **RandomizedSearchCV**, **GridSearchCV**, and (bonus) **Optuna**.
- Evaluate on a clean test set and interpret **feature importances** (permutation).

> 📘 Inspired by Aurélien Géron, *Hands‑On Machine Learning*, Ch. 2 (pipelines, CV) & Ch. 7 (model tuning).


## 0) Imports & Setup

In [1]:

import pandas as pd
import numpy as np
import datetime
from pathlib import Path

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from google.colab import files

import joblib
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


## 1) Load the cleaned dataset (from Day 03)

In [ ]:
uploaded = files.upload()

df = pd.read_csv("vehicles_cleaned.csv")
print("shape:", df.shape)
df.head()



## 2) Train/Test Split (hold‑out first) + Target transform

- Keep a **test set** aside to estimate real‑world performance (Géron: avoid peeking!).  
- Vehicle prices are often **right‑skewed** → we model **log(price+1)** for stability, then convert back.


In [ ]:

# Optional: create price buckets to stratify (comment out if it errors for tiny datasets)
df = df.copy()
df['price_cat'] = pd.cut(df['price'],
                         bins=[0, 5000, 15000, 30000, 60000, np.inf],
                         labels=[1,2,3,4,5])

# Train/test split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=RANDOM_STATE, stratify=df['price_cat'])

for s in (train_set, test_set):
    s.drop(columns=['price_cat'], inplace=True)

# Target on log scale
y_train = np.log1p(train_set['price'])
X_train = train_set.drop(columns=['price'])

y_test = np.log1p(test_set['price'])
X_test  = test_set.drop(columns=['price'])

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)



## 3) Light Feature Engineering (in code, not by hand)

Keep transformations **inside the pipeline** where possible.  
Here we only craft a simple, deterministic feature: **`car_age = current_year - year`**.


In [ ]:

current_year = datetime.datetime.now().year

def add_basic_features(X: pd.DataFrame) -> pd.DataFrame:
    X = X.copy()
    if 'year' in X.columns:
        X['car_age'] = current_year - X['year']
        X = X.drop(columns=['year'])
    return X

X_train = add_basic_features(X_train)
X_test  = add_basic_features(X_test)

X_train.head()



## 4) Preprocessing Pipelines (Géron Ch. 2)

- **Numeric**: `SimpleImputer(median)` → `StandardScaler`  
- **Categorical**: `SimpleImputer(most_frequent)` → `OneHotEncoder(handle_unknown="ignore")`  
- Combine via `ColumnTransformer` for one unified preprocessing step.


In [ ]:

numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocess = ColumnTransformer([
    ("num", num_pipe, numeric_features),
    ("cat", cat_pipe, categorical_features),
])

numeric_features, categorical_features


### Helper: consistent cross‑validation evaluation (on log target)

In [ ]:

from sklearn.model_selection import KFold

def cv_rmse(model, X, y, cv=5):
    scores = cross_val_score(model, X, y, scoring="neg_root_mean_squared_error", cv=cv, n_jobs=-1)
    return -scores.mean(), -scores.std()

CV = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
CV


## 5) Baseline model comparison (no tuning yet)

In [ ]:

from collections import OrderedDict

candidates = OrderedDict({
    "LinearRegression": LinearRegression(),
    "DecisionTree": DecisionTreeRegressor(random_state=RANDOM_STATE),
    "RandomForest": RandomForestRegressor(random_state=RANDOM_STATE, n_estimators=200, n_jobs=-1),
    "HistGradientBoosting": HistGradientBoostingRegressor(random_state=RANDOM_STATE, early_stopping=True),
})

results = []

for name, est in candidates.items():
    pipe = Pipeline([
        ("prep", preprocess),
        ("model", est)
    ])
    mean_rmse, std_rmse = cv_rmse(pipe, X_train, y_train, cv=CV)
    results.append({"model": name, "cv_rmse_log": mean_rmse, "cv_rmse_log_std": std_rmse})

baseline_df = pd.DataFrame(results).sort_values("cv_rmse_log")
baseline_df



## 6) Hyperparameter Tuning (Géron Ch. 7)

- Use **RandomizedSearchCV** for a broad, cheap scan.  
- Refine with **GridSearchCV** on a narrower region if needed.  
- We’ll tune **RandomForest** and **DecisionTree**. We’ll also try **HistGradientBoosting** (fast, strong baseline).


### 6.1 Decision Tree — GridSearchCV

In [ ]:

dt_pipe = Pipeline([
    ("prep", preprocess),
    ("model", DecisionTreeRegressor(random_state=RANDOM_STATE))
])

dt_param_grid = {
    "model__max_depth": [None, 5, 10, 20, 30],
    "model__min_samples_split": [2, 5, 10, 20],
    "model__min_samples_leaf": [1, 2, 4, 8]
}

dt_grid = GridSearchCV(dt_pipe, dt_param_grid, cv=CV, scoring="neg_root_mean_squared_error", n_jobs=-1)
dt_grid.fit(X_train, y_train)

dt_best = dt_grid.best_estimator_
dt_best_score = -dt_grid.best_score_

print("DT best params:", dt_grid.best_params_)
print("DT CV RMSE (log):", dt_best_score)


### 6.2 Random Forest — RandomizedSearchCV

In [ ]:

from scipy.stats import randint, uniform

rf_pipe = Pipeline([
    ("prep", preprocess),
    ("model", RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1))
])

rf_param_dist = {
    "model__n_estimators": randint(150, 500),
    "model__max_depth": [None, 10, 20, 30],
    "model__min_samples_split": randint(2, 20),
    "model__min_samples_leaf": randint(1, 10),
    "model__max_features": ["auto", "sqrt", "log2"],
}

rf_rand = RandomizedSearchCV(
    rf_pipe, rf_param_dist,
    n_iter=30, cv=CV, random_state=RANDOM_STATE,
    scoring="neg_root_mean_squared_error", n_jobs=-1, verbose=0
)
rf_rand.fit(X_train, y_train)

rf_best = rf_rand.best_estimator_
rf_best_score = -rf_rand.best_score_

print("RF best params:", rf_rand.best_params_)
print("RF CV RMSE (log):", rf_best_score)


### 6.3 HistGradientBoosting — RandomizedSearchCV (fast, strong)

In [ ]:

hgb_pipe = Pipeline([
    ("prep", preprocess),
    ("model", HistGradientBoostingRegressor(random_state=RANDOM_STATE, early_stopping=True))
])

hgb_param_dist = {
    "model__max_depth": [None, 3, 5, 7, 10],
    "model__learning_rate": uniform(0.01, 0.3),
    "model__l2_regularization": uniform(0.0, 0.5),
    "model__max_iter": randint(100, 500),
    "model__min_samples_leaf": randint(10, 100),
}

hgb_rand = RandomizedSearchCV(
    hgb_pipe, hgb_param_dist,
    n_iter=30, cv=CV, random_state=RANDOM_STATE,
    scoring="neg_root_mean_squared_error", n_jobs=-1, verbose=0
)
hgb_rand.fit(X_train, y_train)

hgb_best = hgb_rand.best_estimator_
hgb_best_score = -hgb_rand.best_score_

print("HGB best params:", hgb_rand.best_params_)
print("HGB CV RMSE (log):", hgb_best_score)



### 6.4 (Bonus) Optuna — smarter search

You can comment this out if you want speed. This tunes **HistGradientBoosting** via cross‑val.


In [ ]:

# %%time
try:
    import optuna
except Exception as e:
    optuna = None
    print("Optuna not found. Install with: pip install optuna")

if optuna is not None:
    def objective(trial):
        params = {
            "model__learning_rate": trial.suggest_float("model__learning_rate", 1e-3, 0.3, log=True),
            "model__max_depth": trial.suggest_categorical("model__max_depth", [None, 3, 5, 7, 10]),
            "model__l2_regularization": trial.suggest_float("model__l2_regularization", 0.0, 0.5),
            "model__max_iter": trial.suggest_int("model__max_iter", 100, 500),
            "model__min_samples_leaf": trial.suggest_int("model__min_samples_leaf", 10, 100),
        }
        pipe = Pipeline([("prep", preprocess),
                         ("model", HistGradientBoostingRegressor(random_state=RANDOM_STATE, early_stopping=True))])
        pipe.set_params(**params)
        scores = cross_val_score(pipe, X_train, y_train, scoring="neg_root_mean_squared_error", cv=CV, n_jobs=-1)
        return -scores.mean()

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=25, show_progress_bar=False)
    print("Optuna best value (CV RMSE log):", study.best_value)
    print("Optuna best params:", study.best_params)
else:
    print("Skipping Optuna section.")


## 7) Pick the best tuned model (lowest CV RMSE on log‑price)

In [ ]:

tuned = [
    ("DecisionTree", dt_best, dt_best_score),
    ("RandomForest", rf_best, rf_best_score),
    ("HistGradientBoosting", hgb_best, hgb_best_score),
]
best_name, best_est, best_cv_rmse_log = sorted(tuned, key=lambda t: t[2])[0]
print("Selected best model:", best_name, "| CV RMSE (log):", best_cv_rmse_log)


## 8) Final evaluation on the untouched test set

In [ ]:

best_est.fit(X_train, y_train)

y_pred_log = best_est.predict(X_test)
test_rmse_log = mean_squared_error(y_test, y_pred_log, squared=False)

# Transform back to original price units
y_true = np.expm1(y_test)
y_pred = np.expm1(y_pred_log)
test_rmse = mean_squared_error(y_true, y_pred, squared=False)
test_r2   = r2_score(y_true, y_pred)

print(f"Test RMSE (log scale): {test_rmse_log:.4f}")
print(f"Test RMSE (price units): {test_rmse:,.2f}")
print(f"Test R2 (price units): {test_r2:.4f}")


## 9) Interpretability — Permutation Importances

In [ ]:

# Get final feature names after ColumnTransformer
def get_feature_names(preprocessor: ColumnTransformer):
    output_features = []
    # numeric
    num_names = preprocessor.named_transformers_['num'].get_feature_names_out(numeric_features)                 if hasattr(preprocessor.named_transformers_['num'], 'get_feature_names_out') else numeric_features
    output_features.extend(list(num_names))
    # categorical
    ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']
    cat_names = ohe.get_feature_names_out(categorical_features)
    output_features.extend(list(cat_names))
    return output_features

prep = best_est.named_steps['prep']
feat_names = get_feature_names(prep)

perm = permutation_importance(best_est, X_test, y_test, n_repeats=5, random_state=RANDOM_STATE, n_jobs=-1, scoring="neg_root_mean_squared_error")
imp_df = pd.DataFrame({
    "feature": feat_names,
    "importance": perm.importances_mean
}).sort_values("importance", ascending=False).head(25)

imp_df


## 10) Save the best model + preprocess pipeline

In [ ]:

out_dir = Path("./day05_artifacts")
out_dir.mkdir(exist_ok=True)

joblib.dump(best_est, out_dir / f"best_model_{best_name}.joblib")
imp_df.to_csv(out_dir / "permutation_importances_top25.csv", index=False)

print("Saved:", list(out_dir.glob("*")))



---

## ✅ What I learned
- Pipelines prevent leakage and guarantee identical transforms for train/test.
- Cross‑validation gives a fairer estimate than a single split.
- RandomizedSearch is efficient for broad scans; refine with GridSearch.
- HistGradientBoosting is a great fast baseline.

## 🔜 Next
- Try **GradientBoostingRegressor**, **XGBoost** or **LightGBM**.
- Calibrate predictions or use **Quantile Regression** for price intervals.
- Log‑target vs. direct‑target comparison.
- Export to **ONNX** / **PMML** or serve via **FastAPI** for deployment.

Happy modeling! 🚀
